In [ ]:
#Temperatur dependent defect document
# General
import numpy as np
import matplotlib.pyplot as plt
from eskild_function import *

# For handling structures and visualizing structures
from ase import Atoms
from ase.build import graphene_nanoribbon
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.io import read, write

# For MD
from ase.calculators.tersoff import Tersoff
from ase.constraints import FixAtoms
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
from ase import units
from ase.neighborlist import neighbor_list

import os
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [ ]:
# --- Structure Definition ---
kind = "armchair"     # "armchair" or "zigzag"
n = 5                 # width parameter
length = 1            # periodic repetitions along z, i.e., transport direction here
vacuum = 15.0         # vacuum in non-periodic directions (Å)
bond = 1.43877067     # Optimized using C.tersoff potential discussed below

ribbon = graphene_nanoribbon(n=n,
                             m=length,
                             type=kind,
                             C_C=bond,
                             vacuum=vacuum)
ribbon.pbc = True
ribbon = sort_atoms(ribbon) # Function from eskild_function
structure = ribbon

# --- Hamiltonian Setup (Pristine) ---
# Create a transport structure with enough length to define leads
Ntransport = 5 
pristine_structure = structure.repeat((1, 1, Ntransport))
pos = pristine_structure.positions

# Sort atoms (Standard sorting for this project)
x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
if len(x_sort_args) > 10:
    x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
pos_sorted = pos[x_sort_args]

# Parameters for splitting
n_lead_atoms = 20 

# Extract Matrices
H_full = hamiltonian(pos_sorted) 

idx_L = np.arange(n_lead_atoms)
idx_R = np.arange(len(pos_sorted)-n_lead_atoms, len(pos_sorted))
idx_D = np.arange(n_lead_atoms, len(pos_sorted)-n_lead_atoms)

H_L = H_full[np.ix_(idx_L, idx_L)]      
H_D = H_full[np.ix_(idx_D, idx_D)]      
H_R = H_full[np.ix_(idx_R, idx_R)]      
t_L = H_full[np.ix_(idx_D[:n_lead_atoms], idx_L)]      
t_R = H_full[np.ix_(idx_D[-n_lead_atoms:], idx_R)]   

# --- MD Setup ---
timestep = 1.0
calc = Tersoff.from_lammps("C.tersoff")

In [ ]:
# Define Hamiltonian with larger cutoff to avoid artificial bond breaking
def hamiltonian(xyz):
    bond = 1.43877067
    Vpppi = -2.7
    # Cutoff adjusted to be physically robust:
    # Includes 1st neighbors (~1.42 A) and thermal stretching.
    # Excludes 2nd neighbors (~2.46 A).
    # 2.24 A is a safe middle ground.
    cut = bond + 0.8
    dist = np.linalg.norm(xyz[None, :, :] - xyz[:, None, :], axis=2)
    with np.errstate(divide='ignore', invalid='ignore'):
        H = np.where((dist < cut) & (dist > 0.1), Vpppi * (bond / dist)**2, 0.0)
    return H

def resistance(Transmission):
    # Resistance in units of 1/G0
    # G = G0 * Transmission
    # R = 1/G = 1/(G0 * Transmission) = (1/Transmission) * (1/G0)
    # Returning value in units of 1/G0 means returning 1/Transmission
    # Add small epsilon to avoid division by zero
    T_safe = np.array(Transmission)
    T_safe[T_safe < 1e-12] = 1e-12
    return 1.0 / T_safe

# Parameters needed for the calculation
# Use astype(int) to ensure we have integers for the loop ranges
ntiles_list = np.linspace(5, 50, 45).astype(int) # Varying lengths (number of unit cell repetitions)

# Simulation parameters
md_temp_K = 300.0
md_nsteps = 500  # Number of MD steps per length (adjust as needed for time vs accuracy)
md_dump = 10     # Interval for saving snapshots
eta = 0.0001
E_fermi = 0.5    # Shifted Energy to hit the band (Try 0.1, 0.2, 0.5 if 0.0 is gapped)

# Pre-compute Self-Energies at Fermi Level
z_fermi = E_fermi + 1j * eta
sig_s_L_fermi, _, _, _ = self_energy_decimation(z_fermi, H_L, t_L, iterations=100)
sig_s_R_fermi, _, _, _ = self_energy_decimation(z_fermi, H_R, t_R, iterations=100)

def calculate_T_at_fermi(H_device, sig_L, sig_R):
    """
    Calculate Transmission at Fermi level for a given Device Hamiltonian
    """
    size = H_device.shape[0]
    I = np.eye(size)
    z = E_fermi + 1j * eta
    
    # Construct Effective Hamiltonian
    device_zero = np.zeros((size, size), dtype=complex)
    gray_left = device_zero.copy()
    
    n_lead = sig_L.shape[0]
    gray_left[:n_lead, :n_lead] = sig_L
    
    gray_right = device_zero.copy()
    n_lead_R = sig_R.shape[0]
    gray_right[-n_lead_R:, -n_lead_R:] = sig_R
    
    H_eff = H_device + gray_left + gray_right
    
    # Gamma matrices
    gamma_L = 1j * (gray_left - np.conjugate(np.transpose(gray_left)))
    gamma_R = 1j * (gray_right - np.conjugate(np.transpose(gray_right)))
    
    # Green's Function
    G_eff = np.linalg.inv(z*I - H_eff)
    
    T = np.trace(gamma_L @ G_eff @ gamma_R @ np.conjugate(np.transpose(G_eff)))
    return np.real(T)

def run_simulation_for_length(ntile):
    """
    Runs MD and analyzes transport for a specific nanoribbon length.
    Returns: (avg_resistance_from_avg_T, mean_of_R_samples)
    """
    # 1. Create Structure for this length
    xyz = structure.positions
    lattice = structure.cell[:]
    tiledir = 2 # z-direction
    
    # Create repeated structure
    for n in range(1, ntile):
        xyz = np.concatenate((xyz, structure.positions + lattice[tiledir, :]*n))
    
    tilemat = np.eye(3, dtype=int)
    tilemat[tiledir, tiledir] = ntile
    lattice_long = tilemat @ lattice
    natoms = len(xyz)
    
    current_md_structure = Atoms(natoms*["C"], positions=xyz, cell=lattice_long, pbc=True)
    
    # Apply Constraints (Leads and Edges)
    natoms_elec = len(structure)
    fixed_uc = 2
    leftinds = list(range(0, natoms_elec*fixed_uc))
    rightinds = list(range(natoms - natoms_elec*fixed_uc, natoms))
    
    cutoff = 1.5
    bulk_nneighbors = 3
    i_list, j_list = neighbor_list("ij", current_md_structure, cutoff)
    counts = np.bincount(i_list, minlength=len(current_md_structure))
    edgeinds = list(np.where(counts < bulk_nneighbors)[0])
    
    allinds = np.unique(leftinds + rightinds + edgeinds)
    current_md_structure.set_constraint(FixAtoms(mask=allinds))
    current_md_structure.calc = calc # Use the Tersoff calculator defined earlier

    # 2. Run MD
    # Use unique filename per process to avoid conflicts
    # Ensure MD_files directory exists
    os.makedirs("MD_files", exist_ok=True)
    temp_xyz_file = f"MD_files/md_T{int(md_temp_K)}_L{ntile}.xyz"
    
    if os.path.exists(temp_xyz_file):
        # pass
        # Check if file seems valid (has content)
         if os.path.getsize(temp_xyz_file) > 100:
            pass
         else:
             print(f"File {temp_xyz_file} seemed empty, re-running.")
             MaxwellBoltzmannDistribution(current_md_structure, temperature_K=md_temp_K)
             dyn = Langevin(current_md_structure, timestep*units.fs, temperature_K=md_temp_K, friction=0.01/units.fs, logfile=None)
             dyn.attach(lambda: write(temp_xyz_file, current_md_structure, append=True), interval=md_dump)
             dyn.run(md_nsteps)
    else:
        MaxwellBoltzmannDistribution(current_md_structure, temperature_K=md_temp_K)
        dyn = Langevin(current_md_structure, timestep*units.fs, temperature_K=md_temp_K, friction=0.01/units.fs, logfile=None)
        dyn.attach(lambda: write(temp_xyz_file, current_md_structure, append=True), interval=md_dump)
        
        # Run simulation
        dyn.run(md_nsteps)
    
    # 3. Analyze Trajectory
    # Use index=":" to read all frames, slicing after reading is safer for file I/O in parallel? 
    # Actually reading everything then slicing is fine.
    try:
        traj = read(temp_xyz_file, index=":")
        # Check for empty trajectory
        if len(traj) == 0:
             return (np.nan, np.nan)
    except Exception as e:
        print(f"Error reading {temp_xyz_file}: {e}")
        return (np.nan, np.nan)
        
    start_analysis_frame = int(len(traj)*0.25)
    traj_analysis = traj[start_analysis_frame:]
    
    if len(traj_analysis) == 0:
        return (np.nan, np.nan)

    trans_samples = []
    
    # Processing frames sequentially inside the parallel task to avoid nested parallelism overhead
    for frame in traj_analysis:
        pos = frame.positions
        x_sort_args = np.lexsort((pos[:, 0], pos[:, 2]))
        
        if len(x_sort_args) > 10:
             x_sort_args[[9, 10]] = x_sort_args[[10, 9]]
             
        pos_sorted = pos[x_sort_args]
        
        # Use simple global function call, now that it's redefined above
        H_full_frame = hamiltonian(pos_sorted)
        n_lead_atoms = 20
        idx_D = np.arange(n_lead_atoms, len(pos_sorted)-n_lead_atoms)
        H_D_frame = H_full_frame[np.ix_(idx_D, idx_D)]
        
        T = calculate_T_at_fermi(H_D_frame, sig_s_L_fermi, sig_s_R_fermi)
        trans_samples.append(T)
    

    R_quantum = 12906.0
    
    # Method 1: <R> = 1 / <T> (Physical average current)
    avg_transmission = np.mean(trans_samples)
    if avg_transmission < 1e-12:
        res_from_avg_T = np.nan
    else:
        res_from_avg_T = (1.0 / avg_transmission) * R_quantum
        
    # Method 2: <R> = <1/T> (Average of instantaneous resistances)
    # This is more sensitive to outliers (T close to 0)
    R_samples = resistance(trans_samples) * R_quantum
    res_avg_instant = np.mean(R_samples)
    
    return (res_from_avg_T, res_avg_instant)

# --- Smart Data Loading & Execution ---

# 1. Define File Path within the cell so it's consistent
output_dir = "Defected_tranmission_data/Resistance_Scaling"
os.makedirs(output_dir, exist_ok=True)
filename = f"Resistance_vs_Length_T{int(md_temp_K)}_E{E_fermi}_Nsteps{md_nsteps}_Kind{kind}_W{n}.csv"
filepath = os.path.join(output_dir, filename)

results_dict = {}

# 2. Check for existing data
if os.path.exists(filepath):
    print(f"Checking for existing data in: {filepath}")
    try:
        # Load existing data. delimiter=',' as saved previously. comments='#' to skip header.
        existing_data = np.loadtxt(filepath, delimiter=",", comments="#")
        if existing_data.size > 0:
            if existing_data.ndim == 1:
                existing_data = existing_data.reshape(1, -1)
            for row in existing_data:
                # row[0] = length, row[1] = res_T, row[2] = res_R
                results_dict[int(row[0])] = (row[1], row[2])
            print(f"Loaded {len(results_dict)} existing data points.")
    except Exception as e:
        print(f"Warning: Could not read existing file ({e}). Will recalculate.")

# 3. Determine which lengths need simulation
ntiles_needed = [t for t in ntiles_list if t not in results_dict]
ntiles_needed = np.unique(ntiles_needed).tolist() # Remove duplicates if any

if ntiles_needed:
    print(f"Calculating for lengths: {ntiles_needed}")
    # Run simulation only for missing lengths
    new_results = Parallel(n_jobs=-1)(
        delayed(run_simulation_for_length)(ntile) for ntile in tqdm(ntiles_needed, desc="Processing Lengths")
    )
    
    # Add new results to dictionary
    for t, res in zip(ntiles_needed, new_results):
        results_dict[t] = res
else:
    print("All requested lengths are already calculated. Skipping simulation.")

# 4. Save COMPLETE dataset back to file (merging old and new)
# Convert dict to sorted list for saving
all_keys = sorted(results_dict.keys())
data_to_save = []
for k in all_keys:
    data_to_save.append([k, results_dict[k][0], results_dict[k][1]])
data_to_save = np.array(data_to_save)

header = f"Length(UnitCells),Result_1_over_avgT(Ohms),Result_avg_1_over_T(Ohms)\nParameters: T={md_temp_K}K, E={E_fermi}eV, Steps={md_nsteps}, Type={kind}, Width={n}"
np.savetxt(filepath, data_to_save, delimiter=",", header=header)
print(f"Updated data saved to: {filepath}")

# 5. Extract data for Plotting (Subset of ntiles_list)
# We only plot what was requested in ntiles_list, even if we have more data in the file
plot_ntiles = []
plot_res_T = []
plot_res_R = []

# Sort ntiles_list just in case
sorted_requested_ntiles = np.sort(np.unique(ntiles_list))

for t in sorted_requested_ntiles:
    if t in results_dict:
        plot_ntiles.append(t)
        val = results_dict[t]
        plot_res_T.append(val[0])
        plot_res_R.append(val[1])

plot_ntiles = np.array(plot_ntiles)
plot_res_T = np.array(plot_res_T)
plot_res_R = np.array(plot_res_R)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(plot_ntiles, plot_res_T, '-o', color='blue', label=r'Current Average $\langle R \rangle = R_Q / \langle T \rangle$')
plt.plot(plot_ntiles, plot_res_R, '-s', color='red', alpha=0.6, label=r'Resistance Average $\langle R \rangle = \langle R_Q / T \rangle$')

plt.xlabel('Length (Unit Cells)')
plt.ylabel(r'Resistance ($\Omega$)')
plt.title(f'Resistance vs Length at T={md_temp_K}K, E={E_fermi}eV')
plt.grid(True)
plt.legend()
plt.show()